In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_validate
import tqdm
import random

In [3]:
from common import get_data, get_freq_data, signal_cyclic_shift, generate_multi_signal

In [4]:
X_origin, y_origin = get_data()

TypeError: get_data() missing 1 required positional argument: 'data_path'

In [ ]:
from sklearn.utils import shuffle

def prepare_data(X_origin, y_origin, tau_range, alpha_range, data_size=1000, to_print=False):    
    X = []
    y = []
    alpha_values = []
    tau_values = []
    for i in range(data_size):
        alpha = random.choice(alpha_range)
        tau = random.choice(tau_range)
        signal = generate_multi_signal(X_origin, y_origin, tau, alpha)['multi_impulse']
        
        X.append(signal)
        y.append(alpha/(1+alpha))
        
        alpha_values.append(alpha)
        tau_values.append(tau)
   
    X = np.array(X)
    y = np.array(y)
    
    if to_print:
        print("X shape:", X.shape)
        print("y shape:", y.shape)
    
    X, y = shuffle(X, y)

#     plt.scatter(alpha_values, tau_values)
#     plt.show()
    
    return X, y

In [ ]:
# alpha_range = np.array([np.around(10**i, decimals=4) for i in np.arange(0, 3.1, 0.01)])

alpha_range = np.arange(1, 1000, 0.1)
tau_range = np.arange(-100, 101, 1)

In [ ]:
X, y = prepare_data(X_origin, y_origin, tau_range, alpha_range, data_size=len(X_origin), to_print=True)
X = get_freq_data(X, freq=1)

In [ ]:
from sklearn.preprocessing import normalize

In [ ]:
plt.title("Alpha distribution")
plt.hist(alpha_range)
plt.show()

In [ ]:
plt.title("Target distribution")
plt.hist(y)
plt.show()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import HuberRegressor
from sklearn import linear_model
from sklearn import tree
from sklearn import svm
from sklearn.linear_model import PassiveAggressiveRegressor
import xgboost as xgb
from catboost import CatBoostRegressor

In [ ]:
def print_scores():
    for model in models:    
        model_name = type(model).__name__
        print("Regressor:", model_name)
    #         stregr = StackingRegressor(regressors=models, meta_regressor=model)
        scoring = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']
        scores = cross_validate(model, X, y, scoring=scoring, cv=5)

        r2_score_mean = scores['test_r2'].mean()
        r2_score_std = scores['test_r2'].std()
        mse_score_mean = -scores['test_neg_mean_squared_error'].mean()
        mse_score_std = scores['test_neg_mean_squared_error'].std()
        mae_score_mean = -scores['test_neg_mean_absolute_error'].mean()
        mae_score_std = scores['test_neg_mean_absolute_error'].std()
        rmse_score_mean = np.sqrt(-scores['test_neg_mean_squared_error']).mean()
        rmse_score_std = np.sqrt(-scores['test_neg_mean_squared_error']).std()

        cur_scores = [(r2_score_mean, r2_score_std), (mse_score_mean, mse_score_std),\
                          (mae_score_mean, mae_score_std), (rmse_score_mean, rmse_score_std)]
    #         print('!!!!!!!!')
    #         print("TEST")
    #         print(models_scores[type(models[0]).__name__]['r2'])
    #         print('!!!!!!!!')
        print("95% confidence interval:")
        for i, metric in enumerate(['r2', 'mse', 'mae', 'rmse'], 0):
    #         models_scores[model_name][metric].append(cur_scores[i])
            print(metric, "score: %0.5f (+/- %0.5f)" % (cur_scores[i][0], 2*cur_scores[i][1]))
        print('----------------------------------')
    print('____________________________________________________________')
    print()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def get_models():
    huber_reg = HuberRegressor(epsilon= 4.927, alpha= 0.00759)
    ridge_reg = linear_model.Ridge(solver='saga', max_iter=5000, alpha= 1)
    lasso_reg = linear_model.Lasso(max_iter=5000, alpha=0.0001, normalize=False)
    dt_reg = tree.DecisionTreeRegressor(min_samples_split=9, min_samples_leaf=6, min_weight_fraction_leaf=0.03, 
                                                                                                 max_features='auto')

    pa_reg = PassiveAggressiveRegressor(C=0.00611, max_iter=2800, tol=5.719679731382862e-05)
    xgb_reg = xgb.XGBRegressor(objective="reg:linear", alpha= 0.01, booster='gbtree', eta= 0.323, gamma=0.189, 
                               reg_lambda=0.48, max_depth=6, verbosity=0)
    return [huber_reg, ridge_reg, lasso_reg, dt_reg,  pa_reg, xgb_reg]

In [ ]:
models = get_models()

In [16]:
print_scores()

Regressor: HuberRegressor
95% confidence interval:
r2 score: 0.14019 (+/- 0.15536)
mse score: 0.14796 (+/- 0.02964)
mae score: 0.28215 (+/- 0.00864)
rmse score: 0.38418 (+/- 0.03824)
----------------------------------
Regressor: Ridge
95% confidence interval:
r2 score: 0.19380 (+/- 0.08051)
mse score: 0.13871 (+/- 0.01723)
mae score: 0.28370 (+/- 0.00701)
rmse score: 0.37226 (+/- 0.02311)
----------------------------------
Regressor: Lasso
95% confidence interval:
r2 score: 0.19161 (+/- 0.07659)
mse score: 0.13906 (+/- 0.01588)
mae score: 0.28576 (+/- 0.00725)
rmse score: 0.37276 (+/- 0.02136)
----------------------------------
Regressor: DecisionTreeRegressor
95% confidence interval:
r2 score: 0.24909 (+/- 0.03277)
mse score: 0.12914 (+/- 0.00779)
mae score: 0.26751 (+/- 0.00425)
rmse score: 0.35932 (+/- 0.01079)
----------------------------------
Regressor: PassiveAggressiveRegressor
95% confidence interval:
r2 score: -0.12813 (+/- 0.48409)
mse score: 0.19407 (+/- 0.08520)
mae score:

### Stacking

In [16]:
from mlxtend.regressor import StackingRegressor


In [ ]:
for i in range(len(models)):
    models = get_models()
    print("Metaregressor:", type(models[i]).__name__)

    stregr = StackingRegressor(regressors=models, meta_regressor=models[i])
    scoring = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']
    scores = cross_validate(stregr, X, y, scoring=scoring, cv=5)

    r2_score_mean = scores['test_r2'].mean()
    r2_score_std = scores['test_r2'].std()
    mse_score_mean = -scores['test_neg_mean_squared_error'].mean()
    mse_score_std = scores['test_neg_mean_squared_error'].std()
    mae_score_mean = -scores['test_neg_mean_absolute_error'].mean()
    mae_score_std = scores['test_neg_mean_absolute_error'].std()
    rmse_score_mean = np.sqrt(-scores['test_neg_mean_squared_error']).mean()
    rmse_score_std = np.sqrt(-scores['test_neg_mean_squared_error']).std()

    cur_scores = [(r2_score_mean, r2_score_std), (mse_score_mean, mse_score_std),\
                      (mae_score_mean, mae_score_std), (rmse_score_mean, rmse_score_std)]
    #         print('!!!!!!!!')
    #         print("TEST")
    #         print(models_scores[type(models[0]).__name__]['r2'])
    #         print('!!!!!!!!')
    print("95% confindence interval:")
    for i, metric in enumerate(['r2', 'mse', 'mae', 'rmse'], 0):
    #         models_scores[model_name][metric].append(cur_scores[i])
        print(metric, "score: %0.5f (+/- %0.5f)" % (cur_scores[i][0], 2*cur_scores[i][1]))
    print('--------------------------------------------------------')

### Freq exp

In [5]:
alpha_values = np.array(range(1, 1000, 5))
tau_values = range(-25, 30, 5)
freq_values = [1, 25, 50, 60, 75, 85, 100]
# alpha_values = np.around(np.arange(0, 100, 2), decimals=2)

In [6]:
len(alpha_values)

200

In [7]:
def calculate_scores(X_origin, alpha_values, tau_values, data_size=100, freq=1):
    scores_dict = {}
    
    X, y = prepare_data(X_origin, tau_values, alpha_values, to_print=False, data_size=len(X_origin))
    model = get_models()[-1]
    X_freq = get_freq_data(X, freq=freq)
    model.fit(X_freq, y)

    
    for tau in tqdm.tqdm(tau_values):
        scores_dict[tau] = dict(zip(alpha_values, np.zeros(len(alpha_values))))
        for alpha in alpha_values:
            X_cur, y_cur = prepare_data(X_origin, [tau], [alpha], data_size=data_size)
            X_cur = get_freq_data(X_cur, freq=freq)
            
            y_pred = model.predict(X_cur)
            scores_dict[tau][alpha] = np.sqrt(mean_squared_error(y_pred, y_cur))
    return scores_dict

In [19]:
def dict_to_arrays(scores_dict):
    x, y, z = [], [], []
    for tau, alpha_dict in scores_dict.items():
        for alpha, score in alpha_dict.items():
            x.append(tau)
            y.append(alpha)
            z.append(score)
            
    return [x, y, z]

In [20]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
# matplotlib.use('Qt4Agg')

def plot_score_3d(scores_dict):
    x, y, z = dict_to_arrays(scores_dict)
    alpha_values = list(scores_dict[list(scores_dict.keys())[0]].keys())
    
    fig1 = matplotlib.pyplot.figure(figsize=(7, 5))
    ax1 = Axes3D(fig1)

    ax1.set_yticks(np.log10(alpha_values[::10]))
    ax1.set_yticklabels(alpha_values[::10])

    ax1.set_xlabel(r"${\tau}$", fontsize=15)
    ax1.set_ylabel(r"${\alpha}$", fontsize=15)
    ax1.set_zlabel('Accuracy', fontsize=15)

#     ax1.view_init(30, 150)
    ax1.plot(10*x, np.log10(10*y), 10*z, 'ro', color='b', linewidth=3)

    for angle in range(0, 180):
        ax1.view_init(30, angle)
        plt.draw()
        plt.pause(.001)

    plt.show()

In [21]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

VMIN = 0
VMAX = 1.6
STEP = 0.2
ORIGIN_FREQ = 5000

def plot_color_map(scores_dict, alpha_values, tau_values, alpha_freq=50, tau_freq=2, freq=1):
    _, _, z = dict_to_arrays(scores_dict)
    z = np.array(z)
    Z = z.reshape((len(tau_values), len(alpha_values)))

    plt.figure(figsize=(5, 5))
    ax = plt.gca()

    plt.title("RMSE log(alpha), frequency = " + str(int(ORIGIN_FREQ/freq)) + ' [MHZ]')
    plt.xlabel("Two signals ratio", fontsize=12)
    plt.ylabel("Delta t [ns]", fontsize=12)
    plt.xticks([0, 50, 100, 150, 199], [1, 250, 500, 750, 1000])
    plt.yticks([tau_freq*i for i in range(len(tau_values[::tau_freq]))], [0.2*el for el in tau_values[::tau_freq]])

#     im = ax.imshow(Z, interpolation='bilinear', cmap='spring', aspect='auto')
    im = ax.imshow(Z, interpolation='bilinear', cmap='spring', aspect='auto', vmin=VMIN, vmax=VMAX)
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.5)

#     plt.colorbar(im, cax=cax)
    cbar = plt.colorbar(im, cax=cax)
    cbar.set_ticks(np.arange(VMIN, VMAX, STEP))
    cbar.set_ticklabels(np.round(np.arange(VMIN, VMAX, STEP), decimals=1))
    plt.show()

In [22]:
import pickle

def save_obj(obj, name ):
    with open('./data/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('./data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
freq_values = [1, 5, 10, 25, 50, 60, 75, 85, 100]

In [ ]:
for freq in freq_values:
    X_origin, y_origin = get_data()
    scores_dict = calculate_scores(X_origin, alpha_values, tau_values, data_size=1000, freq=freq)
    plot_color_map(scores_dict, alpha_values, tau_values, freq=freq)
    save_obj(scores_dict, 'scores_freq=' + str(freq))
#     plot_score_3d(scores_dict)

 73%|███████▎  | 8/11 [14:17<05:21, 107.18s/it]

### -----------------------------

In [ ]:
model = StackingRegressor(regressors=models, meta_regressor=models[0])
model.fit(X, y)

In [ ]:
alpha_scores = []
alpha_test_range = np.arange(1, 1002, 1)

In [ ]:
for alpha in alpha_test_range:
    X_cur, y_cur = prepare_data(X_origin, tau_range, [alpha], data_size=1000, to_print=False)
    y_pred = model.predict(X_cur)
    alpha_scores.append(mean_absolute_error(y_pred, y_cur))

In [ ]:
plt.title("Alpha train log distribution")
plt.ylabel("MAE")
plt.xlabel("Alpha")
plt.plot(alpha_scores)
plt.show()